by [@tozCSS](https://twitter.com/tozCSS)

<img style="float: right;" src="data/vtr.png">
This notebook contains some maps and plots regarding the last two general election results in Turkey (held in Jun 7, 2015 and Nov 1, 2015).

I also estimate the vote transition rates (the image on the right) using a recursive method developed by [Andreadis](http://www.tandfonline.com/doi/abs/10.1080/17457280902799089) (An R script and the instructions are also made [available](http://www.polres.gr/en/vtr)). To get the exact transition values you can mouse-over the links in the interactive Sankey diagram at the bottom of this notebook. 

Finally, I respond to the fraud claims due to the high increase in the number of registered voters (and valid votes) in 5 months by looking at the correlation between the changes in party shares and the valid votes.

## Data Source
Both Jun 7 and Nov 1 election results are scraped from the Nov 1, 2015 [election results page](http://www.yenisafak.com/secim-2015-kasim/secim-sonuclari) of the Yenisafak daily. Scraped datasets are available in two files as [Nov 1,2015](data/nov.csv) and [Jun 7,2015](data/jun.csv) in csv format.

[UPDATE] : Town level data added. [Nov 1,2015](data/nov_towns.csv) and [Jun 7,2015](data/jun_towns.csv) 

## Election Results Maps
I created maps for the last two general election results of Turkey for four parties with different legends. If you click on any of the cities in those maps (below), vote shares of the four parties are shown for that particular election.

The colors in these maps represent the party vote share in those cities. Since one of the more interesting questions regarding the Nov 1, 2015 election is in which cities AKP increased its votes, I also created its Jun 7, 2015 election density map to facilitate us seeing the change.

## Vote Share Scatter Plots of Each Party per City for the Last Two Elections
For all of the parties, their vote shares in the last two elections is compared in a scatter plot where the dots represent the cities and their colors represent the geographic region they belong to. One interesting pattern for example is that unlike other parties, CHP increased its vote share in some cities and lost some in others. On the other hand we see that AKP increased its vote share in every city, whereas MHP and HDP lost their shares almost in every single city.

In [1]:
import json
import pandas as pd
from unidecode import unidecode
import folium
from IPython.display import HTML
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
#read the data
nov = pd.read_csv('data/nov.csv')
jun = pd.read_csv('data/jun.csv')

In [3]:
def create_map(is_nov,threshold_scale,fill_color,party):
    mapname = party+'_share_nov' if is_nov else party+'_share_jun'
    electiondate = 'Nov 1, 2015' if is_nov else 'Jun 7, 2015'
    tr_geo = 'tr_nov.geojson' if is_nov else 'tr_jun.geojson'
    df = nov if is_nov else jun
    tr = folium.Map(location=[39.5, 35], zoom_start=6, tiles='Mapbox Bright')
    tr.geo_json(geo_path=tr_geo, data=df, data_out=mapname+'.json',
                columns=['city', party],
                threshold_scale=threshold_scale,
                key_on='feature.properties.name',
                fill_color=fill_color, fill_opacity=0.7, line_opacity=0.2,
                legend_name=party+' Vote Shares (%), '+electiondate+' Turkish General Elections')
    tr.create_map(path=mapname+'.html')
    return HTML('<iframe src='+mapname+'.html style="width: 100%; height: 500px; border: none"></iframe>')

In [4]:
# AKP Vote Shares (%), Nov 1, 2015 Turkish General Elections
create_map(True,[15,25,35,45,55,65],'YlOrRd','AKP')

In [5]:
# Vote Shares (%), Jun 7, 2015 Turkish General Elections
create_map(False,[15,25,35,45,55,65],'YlOrRd','AKP')

In [6]:
# CHP Vote Shares (%), Nov 1, 2015 Turkish General Elections
create_map(True,[10,15,20,25,30,35],'PuRd','CHP')

In [7]:
# CHP Vote Shares (%), Jun 7, 2015 Turkish General Elections
create_map(False,[10,15,20,25,30,35],'PuRd','CHP')

In [8]:
# MHP Vote Shares (%), Nov 1, 2015 Turkish General Elections
create_map(True,[5,10,15,20,25,30],'GnBu','MHP')

In [9]:
# MHP Vote Shares (%), Jun 7, 2015 Turkish General Elections
create_map(False,[5,10,15,20,25,30],'GnBu','MHP')

In [10]:
# HDP Vote Shares (%), Nov 1, 2015 Turkish General Elections
create_map(True,[5,10,15,20,25,30],'BuPu','HDP')

In [11]:
# HDP Vote Shares (%), Jun 7, 2015 Turkish General Elections
create_map(False,[5,10,15,20,25,30],'BuPu','HDP')

In [12]:
#Scatter plot
import plotly.plotly as py
from plotly.graph_objs import *
from palettable.colorbrewer.qualitative import Dark2_7 as colmap

il = pd.read_csv('data/city_meta.csv',usecols=['il','bolge'])
colors = dict(zip(il['bolge'].unique().tolist(),colmap.hex_colors))
parties = ['AKP', 'CHP','MHP','HDP']
jun['bolge'] = il['bolge']
nov['bolge'] = il['bolge']

for p in parties:
    lim = max(nov[p].max(),jun[p].max())
    #each region is a trace, otherwise they do not show up in the legend
    traces = [Scatter(x=jun[jun.bolge==b][p],y=nov[nov.bolge==b][p],
                      mode='markers', text=il[il.bolge==b]['il'], name=b,
                      marker=Marker(color=v)) for b,v in colors.items()]
    traces.append(Scatter(x=[0,lim],y=[0,lim],mode='lines', name='y=x',
                        line=Line(color='grey',dash='dash')))
    data = Data(traces)
    layout = Layout(title=p+' Nov 1, 2015 vs Jun 7, 2015 Vote Shares',
                    autosize=True,
                    xaxis=XAxis(title=p+" Jun 7, 2015 Vote Shares",zeroline=False),
                    yaxis=YAxis(title=p+" Nov 1, 2015 Vote Shares",zeroline=False),
                    legend=Legend(x=.01,y=1))
    fig = Figure(data=data,layout=layout)
    url = py.plot(fig,filename= p+' Nov 1, 2015 vs Jun 7, 2015 Vote Shares')

In [13]:
HTML("""<div><a href="https://plot.ly/~toz/828/" target="_blank" title="AKP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="display: block; text-align: center;"><img src="https://plot.ly/~toz/828.png" alt="AKP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a><script data-plotly="toz:828"  src="https://plot.ly/embed.js" async></script></div>""")

In [14]:
HTML("""<div><a href="https://plot.ly/~toz/830/" target="_blank" title="CHP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="display: block; text-align: center;"><img src="https://plot.ly/~toz/830.png" alt="CHP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a><script data-plotly="toz:830"  src="https://plot.ly/embed.js" async></script></div>""")

In [15]:
HTML("""<div><a href="https://plot.ly/~toz/832/" target="_blank" title="MHP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="display: block; text-align: center;"><img src="https://plot.ly/~toz/832.png" alt="MHP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a><script data-plotly="toz:832"  src="https://plot.ly/embed.js" async></script></div>""")

In [16]:
HTML("""<div><a href="https://plot.ly/~toz/834/" target="_blank" title="HDP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="display: block; text-align: center;"><img src="https://plot.ly/~toz/834.png" alt="HDP Nov 1, 2015 vs Jun 7, 2015 Vote Shares" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a><script data-plotly="toz:834"  src="https://plot.ly/embed.js" async></script></div>""")

In [17]:
HTML("""<iframe width="50%" height="450" src="//jsfiddle.net/v5rw0y75/39/embedded/result" allowfullscreen="allowfullscreen" frameborder="0"></iframe>""")

In [18]:
print('Total increase in the registered voters:',nov.registered.sum() - jun.registered.sum())
print('Total increase in the valid votes:',nov.valid.sum() - jun.valid.sum())

Total increase in the registered voters: 1555664.0
Total increase in the valid votes: 1769286.0


Some claimed that it is not normal to have such high number of registered voters (or valid votes) increase in five months. It smells fraud to many. However, I couldn't find any support to these claims in the city level data: there is no significant positive or negative correlation between the changes in the valid votes and changes in any of the party shares. [See the output of the following cell]

In [19]:
#check the correlation of change in registered voters with the change in the party shares
parties = ['AKP', 'CHP','MHP','HDP','others']
change = nov[parties] - jun[parties]
change['registered'] = (nov['registered'] - jun['registered'])/nov['registered']
change.index = nov['city']
change.corr() #change in registered voters vs change in party shares

,AKP,CHP,MHP,HDP,others,registered
AKP,1.000000,-0.250031,-0.345152,-0.348651,-0.170495,-0.120211
CHP,-0.250031,1.000000,0.004348,-0.253937,-0.030682,0.002432
MHP,-0.345152,0.004348,1.000000,-0.465819,-0.334473,0.215103
HDP,-0.348651,-0.253937,-0.465819,1.000000,0.061552,0.049348
others,-0.170495,-0.030682,-0.334473,0.061552,1.000000,-0.156149
registered,-0.120211,0.002432,0.215103,0.049348,-0.156149,1.000000
